<a href="https://colab.research.google.com/github/sheetalojha/GNN-Graph-Classification-AIDS-data/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dgl

     |████████████████████████████████| 4.4 MB 8.2 MB/s 


In [2]:
from dgl.data import LegacyTUDataset
from dgl.dataloading import GraphDataLoader
dataset = LegacyTUDataset('ENZYMES')
num_labels = dataset.num_labels
dataloader = GraphDataLoader(dataset, batch_size=1, shuffle=True)

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


Extracting file to /root/.dgl/ENZYMES


In [3]:
import dgl
import dgl.nn.pytorch as dglnn
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes):
        super(Classifier, self).__init__()
        self.conv1 = dglnn.GraphConv(in_dim, hidden_dim)
        self.conv2 = dglnn.GraphConv(hidden_dim, hidden_dim)
        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, g, h):
        # Apply graph convolution and activation.
        h = F.relu(self.conv1(g, h))
        h = F.relu(self.conv2(g, h))
        with g.local_scope():
            g.ndata['h'] = h
            # Calculate graph representation by average readout.
            hg = dgl.mean_nodes(g, 'h')
            return self.classify(hg)

In [4]:
import torch
import torch.nn.functional as F
model = Classifier(18, 10, num_labels)
opt = torch.optim.Adam(model.parameters())

for epoch in range(20):
    model.train()
    print(epoch)
    for g, labels in dataloader:
        g = dgl.add_self_loop(g)
        feats = g.ndata['feat']
        logits = model(g,feats)
        loss = F.cross_entropy(logits, labels)
        opt.zero_grad()
        loss.backward()
        opt.step()
        print(loss)

0


/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Streaming output truncated to the last 5000 lines.
tensor(1.7612, grad_fn=<NllLossBackward0>)
tensor(1.6285, grad_fn=<NllLossBackward0>)
tensor(1.7261, grad_fn=<NllLossBackward0>)
tensor(1.6337, grad_fn=<NllLossBackward0>)
tensor(1.6299, grad_fn=<NllLossBackward0>)
tensor(1.8266, grad_fn=<NllLossBackward0>)
tensor(2.1524, grad_fn=<NllLossBackward0>)
tensor(1.7100, grad_fn=<NllLossBackward0>)
tensor(0.1768, grad_fn=<NllLossBackward0>)
tensor(0.8876, grad_fn=<NllLossBackward0>)
tensor(2.1119, grad_fn=<NllLossBackward0>)
tensor(2.1216, grad_fn=<NllLossBackward0>)
tensor(1.8735, grad_fn=<NllLossBackward0>)
tensor(1.8057, grad_fn=<NllLossBackward0>)
tensor(1.1586, grad_fn=<NllLossBackward0>)
tensor(2.0280, grad_fn=<NllLossBackward0>)
tensor(1.5797, grad_fn=<NllLossBackward0>)
tensor(1.5984, grad_fn=<NllLossBackward0>)
tensor(1.2135, grad_fn=<NllLossBackward0>)
tensor(2.0795, grad_fn=<NllLossBackward0>)
tensor(1.7056, grad_fn=<NllLossBackward0>)
tensor(1.9767, grad_fn=<NllLossBackward0>)
ten